Un solo Job que cuenta tanto retadores como retados
La clave de salida es una tupla (TYPE, player_id) donde TYPE es 'RETADOR' o 'RETADO'
El valor de salida es la cantidad de veces que ese player_id aparece como retador o retado

In [ ]:
from MRE import Job
inputDir = "./jugadores/"
outputDir = "./retadoresContados/"

In [ ]:
def fmap(key, value, context):
    # por cada linea genero dos con clave tipo + id y valor 1
    retador = int(key)
    parts = value.split('	')
    context.write(('RETADOR', retador), 1)
    retado = int(parts[0])
    context.write(('RETADO', retado), 1)

def fcombiner(key, values, context):
    # simple combiner: sum counts locally
    s = 0
    for v in values:
        s += int(v)
    context.write(key, s)

def fred(key, values, context):
    # reducer to sum counts per (TYPE, player_id)
    s = 0
    for v in values:
        s += int(v)
    # write the composite key (TYPE, player) and the total count
    context.write(key, s)

jobCountBoth = Job(inputDir, outputDir, fmap, fred)
jobCountBoth.setCombiner(fcombiner)
success = jobCountBoth.waitForCompletion()

print(success)

El segundo job recibe la salida del primer job del tipo: TYPE	player	count

In [ ]:
inputDir = "./retadoresContados/"
outputDir = "./output2/"

In [ ]:
def fmap_max(key, value, context):
    context.write(key, value)

def fred_max(key, values, context):
    max_count = -1
    max_player = None
    for v in values:
        parts = v.split('	')
        player = int(parts[0])
        cnt = int(parts[1])
        if cnt > max_count:
            max_count = cnt
            max_player = player
    context.write(key, (max_player, max_count))

jobMax = Job(inputDir, outputDir, fmap_max, fred_max)
success = jobMax.waitForCompletion()

print(success)